<h1>CS4619: Artificial Intelligence II</h1>
<h1>Genetic Algorithms</h1>
<h2>
    Derek Bridge<br>
    School of Computer Science and Information Technology<br>
    University College Cork
</h2>

<h1>Initialization</h1>
$\newcommand{\Set}[1]{\{#1\}}$ 
$\newcommand{\Tuple}[1]{\langle#1\rangle}$ 
$\newcommand{\v}[1]{\pmb{#1}}$ 
$\newcommand{\cv}[1]{\begin{bmatrix}#1\end{bmatrix}}$ 
$\newcommand{\rv}[1]{[#1]}$ 
$\DeclareMathOperator{\argmax}{arg\,max}$ 
$\DeclareMathOperator{\argmin}{arg\,min}$ 
$\DeclareMathOperator{\dist}{dist}$
$\DeclareMathOperator{\abs}{abs}$

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<h1>Evolving Table-Driven Agents</h1>
<ul>
    <li>Previously, we programmed the agent: by filling in the table.</li>
    <li>Today, we will <b>evolve</b> the agent.</li>
    <li>To make the problem harder:
        <ul>
            <li>&hellip;we will now allow all 8 sensors <img src="images/eight_sensors.png" /></li>
            <li>&hellip;and 4 actions
                <ul>
                    <li>MOVE</li>
                    <li>TURN(RIGHT, 2)</li>
                    <li>TURN(RIGHT, 4)</li>
                    <li>TURN(LEFT, 2)</li>
                </ul>
            </li>
        </ul>
    </li>
</ul>

<h1>Evolution by Natural Selection</h1>
<ul>
    <li><b>Inheritance:</b> Individuals inherit characteristics from their parents.</li>
    <li><b>Variation:</b> But individuals vary&hellip;
        <ul>
            <li>&hellip; by combining and mutating inherited characteristics.</li>
        </ul>
    </li>
    <li><b>Fitness:</b> Lower fitness characteristics are selectively eliminated (<i>'survival of the fittest'</i>).</li>
</ul>

<h1>Genetic Algorithms (GAs)</h1>
<ul>
    <li>GAs emulate evolution.</li>
    <li>They iteratively update a <b>population</b> of <b>individuals</b>.</li>
    <li>Each new generation of the population is obtained by probabilistically selecting fitter individuals
        from the current generation.
        <ul>
            <li>Some of these individuals are <b>copied</b> to the next generation unchanged.</li>
            <li>Some are subject to <b>crossover</b> to create new offspring.</li>
            <li>Some of the new generation undergo <b>mutation</b>.</li>
        </ul>
    </li>
    <li>GAs differ from real-world evolution, e.g. each generation is the same size as the previous one.</li>
</ul>

<h1>A genetic algorithm</h1>
<ul>
    <li>$n$ is the number of individuals in the population.</li>
    <li>$\chi$ is the proportion of the population to be replaced by crossover, e.g. 0.8.</li>
    <li>$\mu$ is the mutation rate, e.g. 0.01.</li>
</ul>
<figure style="border: 1px solid black; background-color: #D0D0D0">
    <figcaption style="border-bottom: 1px solid black">
        GA($n, \chi, \mu$)
    </figcaption>
    <ul>
        <li>// Initialise generation 0:<br />
            $k = 0$;<br />
            $P_k =$ a population of 𝑛 randomly-generated individuals;
        </li>
        <li>// Evaluate $P_k$:<br />
            Compute $\mathit{fitness}(i)$ for each $i \in P_k$;
        </li>
        <li>do {
            <ul>
                <li>// Create generation $k+1$:<br />
                    // 1. Copy:<br />
                    Select $(1 - \chi) \times n$ members of $P_k$ and insert into $P_{k+1}$;<br />
                    // 2. Crossover:<br />
                    Select $\chi \times n$ members of $P_k$;<br />
                    pair them up;<br />
                    produce offspring;<br /> 
                    insert the offspring into $P_{k+1}$;<br />
                    // 3. Mutate:<br />
                    Select $\mu \times n$ members of $P_{k+1}$;<br />
                    invert a randomly-selected bit in each;
                </li>
                <li>// Evaluate $P_{k+1}$:<br />
                    Compute $\mathit{fitness}(i)$ for each $i \in P_{k+1}$;
                </li>
                <li>// Increment:<br />
                    $k = k + 1$;
                </li>
            </ul>
            } while fitness of fittest individual in $P_k$ is not high enough;
        </li>
        <li>return the fittest individual from $P_k$;
        </li>
    </ul>
</figure>

<h1>Representation of individuals</h1>
<ul>
    <li>Individuals are represented by <em>bit strings</em>.</li>
    <li>This requires a way of <b>encoding</b> and <b>decoding</b>.</li>
</ul>

<h1>Encoding/decoding</h1>
<ul>
    <li>Suppose this is the agent's table:
        <table>
            <tr><th style="border: 1px solid black;">Percept</th><th style="border: 1px solid black;">Action</th></tr>
            <tr><td style="border: 1px solid black;">00000000</td><td style="border: 1px solid black;">MOVE</td></tr>
            <tr><td style="border: 1px solid black;">00000001</td><td style="border: 1px solid black;">TURN(LEFT, 2)</td></tr>
            <tr><td style="border: 1px solid black;">00000010</td><td style="border: 1px solid black;">TURN(RIGHT, 4)</td></tr>
            <tr><td style="border: 1px solid black;">$\vdots$</td><td style="border: 1px solid black;">$\vdots$</td></tr>
        </table>   
    </li>
    <li>We can assign unique codes to the actions:
        <table>
            <tr><th style="border: 1px solid black;">Action</th><th style="border: 1px solid black;">Bit string</th></tr>
            <tr><td style="border: 1px solid black;">MOVE</td><td style="border: 1px solid black;">00</td></tr>
            <tr><td style="border: 1px solid black;">TURN(RIGHT, 2)</td><td style="border: 1px solid black;">01</td></tr>
            <tr><td style="border: 1px solid black;">TURN(RIGHT, 4)</td><td style="border: 1px solid black;">10</td></tr>
            <tr><td style="border: 1px solid black;">TURN(LEFT, 2)</td><td style="border: 1px solid black;">11</td></tr>
        </table>
    <li>We can concatenate all entries in the table to form one long bit string:
        <div style="text-align: center">
            <span style="color: red">00000000</span><span style="color: green">00</span>
            <span style="color: red">00000001</span><span style="color: green">11</span>
            <span style="color: red">00000010</span><span style="color: green">10</span>
            &hellip;
        </div>
    </li>
    <li>Class exercise: How long will this bit string be?</li>
    <li>In fact, we don't need to include the percepts:
        <div style="text-align: center">
            <span style="color: green">00</span>
            <span style="color: green">11</span>
            <span style="color: green">10</span>
            &hellip;
        </div>
    </li>
    <li>Class exercise: How long will this bit string be now?
    <li>Class exercise: How many different bit strings (or tables or agents) are there?</li>
</ul>

<h1>Fitness</h1>
<ul>
    <li>The GA needs a (task-specific) <b>fitness function</b>.</li>
    <li>E.g. place an individual into a room then, of all the cells that it visits, calculate the proportion that 
        are adjacent to walls.
    </li>
    <li>Typically, <em>average</em> performance over <em>several tasks</em> is computed.</li>
</ul>

<h1>Copy</h1>
<ul>
    <li>How do we select the $(1 - \chi) \times n$ individuals who will be copied over?</li>
    <li>Obviously, influenced by their fitness, but we don't simply take the <em>fittest</em>.</li>
    <li>Instead, it is probabilistic, e.g.:
        <ul>
            <li><b>Roulette wheel selection</b>:
                <ul>
                    <li>Probability of selection is equal to relative fitness:
                        $$\mathit{Prob}(\mathit{choice} = i) = \frac{\mathit{fitness}(i)}{\sum_{j=1}^n \mathit{fitness}(j)}$$
                    </li>
                </ul>
			</li>
            <li><b>Rank selection</b>:
                <ul>
                    <li>Probability of selection is inversely proportional to position in the population after sorting 
                        by fitness.
                    </li>
                </ul>
            </li>
            <li><b>Tournament selection</b>:
                <ul>
                    <li>Repeatedly, select a random subset of the population and chose the fittest in this subset.</li>
                </ul>
            </li>
        </ul>
    </li>
    <li>Selection is usually done <em>with replacement</em>: an individual can be picked more than once.</li>
</ul>

<h1>Crossover</h1>
<ul>
    <li>In crossover:
        <ul>
            <li>Select $\chi \times n$ individuals.
                <ul>
                    <li>How? By roulette wheel, rank or tournament selection.</li>
                </ul>
            </li>
            <li>Pair them up, giving $(\chi \times n)/2$ pairs.</li>
            <li>Swap a random portion of the father with a random portion of the mother, giving two new offspring.</li>
        </ul>
    </li>
    <li>The offspring may or may not be fitter than their parents:
        <ul>
            <li>We hope roulette wheel/rank/tournament seletion has chosen reasonably fit parents, and the offspring
                might have some fitness advantage by incorporating parts of these parents.
            </li>
            <li>On the other hand, no guarantees!</li>
        </ul>
    </li>
    <li><b>Single-point crossover</b>: choose a random position and swap.</li>
    <li>(There are other forms of crossover, e.g. two-point crossover, uniform crossover &mdash; not covered in
        CS4619.)
    </li>
</ul>

<h1>Single-point crossover</h1>
<figure>
    <img src="images/crossover1.png" />
</figure>

<h1>Efficient single-point crossover</h1>
<ul>
    <li>Generate two masks, e.g.:
        $$\mathit{mask}_1: 11111100000$$
        $$\mathit{mask}_2: 00000011111$$
    </li>
    <li>Then<br />
        $$\mathit{offspring}_1 = (\mathit{parent}_1 \land \mathit{mask}_1) \vee (\mathit{parent}_2 \land \mathit{mask}_2)$$
        $$\mathit{offspring}_2 = (\mathit{parent}_1 \land \mathit{mask}_2) \vee (\mathit{parent}_2 \land \mathit{mask}_1)$$
    </li>
</ul>

<h1>Example of efficient single-point crossover</h1>
<figure>
    <img src="images/crossover4.png" />
</figure>

<h1>Mutate</h1>
<ul>
    <li>Select $\mu \times n$ individuals from the <em>new</em> generation.
        <ul>
            <li>How? Random with <em>uniform</em> probability, not by fitness.</li>
        </ul>
    </li>
    <li>For each selected individual, a bit is chosen at random and this bit is inverted.</li>
    <li>E.g. <img src="images/mutation1.png" /></li>
</ul>

<h1>Efficient mutation</h1>
<ul>
    <li>Generate a mask, e.g.:
        $$\mathit{mask}: 00000010000$$
    </li>
    <li>Then
        $$\mathit{new individual} = \mathit{individual} \oplus \mathit{mask}$$
        where $\oplus$ is exclusive-or.
    </li>
    <li>E.g. <img src="images/mutation2.png" />
    </li>
</ul>

<h1>Discussion</h1>
<ul>
    <li>There’s a risk of <b>crowding</b>:
        <ul>
            <li>This is where a fit individual reproduces a lot and it (or minor variants of it) dominate the population.</li>
            <li>It results in a lack of diversity and possible stagnation.</li>
        </ul>
    </li>
    <li>How to overcome crowding:
        <ul>
            <li>Mutation;</li>
            <li>Rank selection or tournament selection;</li>
            <li>&hellip;</li>
        </ul>
    </li>
</ul>

<h1>Applications</h1>
<ul>
    <li>GAs have been used to evolve:
        <ul>
            <li>Digital circuits</li>
            <li>Factory schedules</li>
            <li>University timetables</li>
            <li>Traffic signal policies</li>
            <li>Neural network architectures</li>
            <li>Similarity measures</li>
            <li>&hellip;</li>
        </ul>
    </li>
    <li>Lecture discussion: How would we do university timetabling using a GA? Are there difficulties?
</ul>